In [21]:
from sigopt import Connection

conn = Connection(client_token="UOZGJVJHBKQZUZQSBUJPOZFCYWPRREGTLSJFKSGOVVEQBRIF")

experiment = conn.experiments().create(
  name="SVM Classifier",
  parameters=[
    dict(
      name="degree",
      bounds=dict(
        min=2,
        max=4
        ),
      type="int"
      ),
    dict(
      name="log_C",
      bounds=dict(
        min=-11.5129,
        max=0
        ),
      type="double"
      ),
    dict(
      name="gamma",
      bounds=dict(
        min=0.000001,
        max=1
        ),
      type="double"
      )
    ],
  metadata=dict(
    template="python_sklearn_svc"
    ),
  observation_budget=30
  )


In [13]:
%pylab
%matplotlib inline
from skimage import io
from glob import glob
from os.path import join
import numpy as np
from skimage.color import gray2rgb
from skimage.transform import resize
from skimage.measure import label, regionprops
from sklearn.externals import joblib
from collections import namedtuple 
import os
import random


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [14]:
path_train='/home/cesar/Desktop/testdataset'
train = glob(join(path_train,'true','*.jpg'))
img_train_labels = [{'imtrue':t, 'imfalse':t.replace('true','falserandom')} for t in train]
train_test_labels=[] #lista de entrenamiento que contiene las etiquetas

dataset =np.zeros((32304,3072), np.uint8)
label= np.zeros((32304),np.uint8)
d=np.zeros((32304),np.uint8)
d
i=0
print(dataset.dtype)
for labels in range(16152):
    train_im_labels=io.imread(img_train_labels[labels]['imtrue'])
    dataset[i] = np.reshape(train_im_labels, (3072))
   
    label[i] = 1
    i = i + 1
    train_false_labels=io.imread(img_train_labels[labels]['imfalse'])
    dataset[i] = np.reshape(train_false_labels, (3072))
    label[i] = 0
    
    i = i + 1

uint8


In [15]:
from skimage.feature import local_binary_pattern as lbp
from skimage.color import rgb2gray
newdataset=np.zeros((dataset.shape[0],8))
for i,v in enumerate(dataset):
    im=dataset[i]
    datasetlbp=lbp(rgb2gray(np.reshape(im,(32,32,3))),8,3,method='uniform')
  
    hist= np.histogram(datasetlbp,normed=True, bins=8, range=(0, 8))
    newdataset[i]=hist[0]

In [ ]:
from sklearn.svm import SVC
clf = SVC(C=80)
clf.fit(newdataset,label)

In [20]:
print(newdataset.shape)
datatraining=newdataset[0:19382]
labeltraining=label[0:19382]
print(datatraining.shape)
print(labeltraining.shape)

(32304, 8)
(19382, 8)
(19382,)


In [22]:
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
import math

def create_model(assignments):
    model = SVC(
        degree=assignments['degree'],
        C=math.exp(assignments['log_C']),
        gamma=assignments['gamma'],
    ).fit(newdataset,label)
    return model

def evaluate_model(assignments):
    model = create_model(assignments)
    return model.score(datatraining, labeltraining)

In [ ]:
for _ in range(experiment.observation_budget):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    assignments = suggestion.assignments
    value = evaluate_model(assignments)

    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value
    )

assignments = conn.experiments(experiment.id).best_assignments().fetch().data[0].assignments

print(assignments)

# This is a SigOpt-tuned model
classifier = create_model(assignments)

Assignments({
  "degree": 3,
  "gamma": 1.0,
  "log_C": 0.0
})
